In [40]:
import os
from dotenv import load_dotenv
import os
from dotenv import load_dotenv, find_dotenv
import warnings
import requests
import json
import time

from utils import mistral
from utils import mistral2

In [41]:
from utils import load_mistral_api_key
api_key, dlai_endpoint = load_mistral_api_key(ret_key=True)

In [42]:
import pandas as pd

In [43]:
data = {
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
df = pd.DataFrame(data)
df

,transaction_id,customer_id,payment_amount,payment_date,payment_status
0,T1001,C001,125.50,2021-10-05,Paid
1,T1002,C002,89.99,2021-10-06,Unpaid
2,T1003,C003,120.00,2021-10-07,Paid
3,T1004,C002,54.30,2021-10-05,Paid
4,T1005,C001,210.20,2021-10-08,Pending


1. Without Function Calling

In [44]:
data = """
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
"""
transaction_id = "T1001"

prompt = f"""
Given the following data, what is the payment status for \
 transaction_id={transaction_id}?

data:
{data}

"""

In [45]:
response = mistral(prompt)
print(response)

To determine the payment status for `transaction_id=T1001`, you need to look at the corresponding entry in the provided data. Here is the relevant snippet for `transaction_id=T1001`:

- `transaction_id`: T1001
- `customer_id`: C001
- `payment_amount`: 125.50
- `payment_date`: 2021-10-05
- `payment_status`: Paid

Therefore, the payment status for `transaction_id=T1001` is **Paid**.


2. With Function Calling

2.1 Creating two Tools

In [46]:
import json

> Tool 1: Retrieve Payment Status

In [47]:
def retrieve_payment_status(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"status": df[df.transaction_id == transaction_id].payment_status.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [48]:
status = retrieve_payment_status(df, transaction_id="T1001")
print(status)

{"status": "Paid"}


In [49]:
type(status)

str

In [50]:
tool_payment_status = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_status",
        "description": "Get payment status of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [51]:
type(tool_payment_status)

dict

> Tool 2: Retrieve Payment Date

In [52]:
def retrieve_payment_date(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"date": df[df.transaction_id == transaction_id].payment_date.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [53]:
date = retrieve_payment_date(df, transaction_id="T1002")
print(date)

{"date": "2021-10-06"}


In [54]:
tool_payment_date = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_date",
        "description": "Get payment date of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [55]:
type(tool_payment_status)

dict

> Creating Tool List

In [56]:
tools = [tool_payment_status, tool_payment_date]

In [57]:
type(tools)

list

In [58]:
tools

[{'type': 'function',
  'function': {'name': 'retrieve_payment_status',
   'description': 'Get payment status of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}},
 {'type': 'function',
  'function': {'name': 'retrieve_payment_date',
   'description': 'Get payment date of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}}]

> Mapping Tools with their Names

In [59]:
import functools

In [60]:
names_to_functions = {
    "retrieve_payment_status": functools.partial(retrieve_payment_status, df=df),
    "retrieve_payment_date": functools.partial(retrieve_payment_date, df=df),
}

In [61]:
names_to_functions["retrieve_payment_status"](transaction_id="T1001")

'{"status": "Paid"}'

In [62]:
tools

[{'type': 'function',
  'function': {'name': 'retrieve_payment_status',
   'description': 'Get payment status of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}},
 {'type': 'function',
  'function': {'name': 'retrieve_payment_date',
   'description': 'Get payment date of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}}]

2.2 User Query

In [63]:
from mistralai.models.chat_completion import ChatMessage
chat_history = [
    ChatMessage(role="user", content="What's the status of my transaction?")
]

2.3 Generate Model Response (User Query with Chat History)

In [64]:
from mistralai.client import MistralClient

model = "mistral-large-latest"

client = MistralClient(api_key=api_key)

response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)

response

ChatCompletionResponse(id='9fb2703a24d446588ac53d0d15bd3893', object='chat.completion', created=1731158575, model='mistral-large-latest', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='Sure! Please provide the transaction ID for me to check the status.', name=None, tool_calls=None), finish_reason=<FinishReason.stop: 'stop'>)], usage=UsageInfo(prompt_tokens=161, total_tokens=175, completion_tokens=14))

In [65]:
response.choices[0].message.content

'Sure! Please provide the transaction ID for me to check the status.'

> Saving Response to the Chat History

In [66]:
chat_history.append(
    ChatMessage(role="assistant", content=response.choices[0].message.content)
)
chat_history.append(ChatMessage(role="user", content="My transaction ID is T1005."))
chat_history

[ChatMessage(role='user', content="What's the status of my transaction?", name=None, tool_calls=None),
 ChatMessage(role='assistant', content='Sure! Please provide the transaction ID for me to check the status.', name=None, tool_calls=None),
 ChatMessage(role='user', content='My transaction ID is T1005.', name=None, tool_calls=None)]

In [67]:
response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)

In [68]:
response

ChatCompletionResponse(id='20ded098d4194f19986df2003e475b17', object='chat.completion', created=1731158577, model='mistral-large-latest', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='XnRb0TBhL', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1005"}'))]), finish_reason=<FinishReason.tool_calls: 'tool_calls'>)], usage=UsageInfo(prompt_tokens=188, total_tokens=218, completion_tokens=30))

> Model Was able to Invoke Right Function Call

In [69]:
response.choices[0].message

ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='XnRb0TBhL', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1005"}'))])

> Appending it to Chat History

In [70]:
chat_history.append(response.choices[0].message)
chat_history

[ChatMessage(role='user', content="What's the status of my transaction?", name=None, tool_calls=None),
 ChatMessage(role='assistant', content='Sure! Please provide the transaction ID for me to check the status.', name=None, tool_calls=None),
 ChatMessage(role='user', content='My transaction ID is T1005.', name=None, tool_calls=None),
 ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='XnRb0TBhL', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1005"}'))])]

2.4 Execute Functions to Obtain the Results

In [71]:
tool_function = response.choices[0].message.tool_calls[0].function
print(tool_function)

name='retrieve_payment_status' arguments='{"transaction_id": "T1005"}'


In [72]:
tool_function.name

'retrieve_payment_status'

In [73]:
tool_function.arguments

'{"transaction_id": "T1005"}'

In [74]:
args = json.loads(tool_function.arguments)
print(args)

{'transaction_id': 'T1005'}


> Manual Function Call (Passing function names and Argument)

In [75]:
function_result = names_to_functions[tool_function.name](**args)
function_result

'{"status": "Pending"}'

> Saving Function Call Result in Chat History

In [76]:
tool_msg = ChatMessage(role="tool", name=tool_function.name, content=function_result)
chat_history.append(tool_msg)
chat_history

[ChatMessage(role='user', content="What's the status of my transaction?", name=None, tool_calls=None),
 ChatMessage(role='assistant', content='Sure! Please provide the transaction ID for me to check the status.', name=None, tool_calls=None),
 ChatMessage(role='user', content='My transaction ID is T1005.', name=None, tool_calls=None),
 ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='XnRb0TBhL', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1005"}'))]),
 ChatMessage(role='tool', content='{"status": "Pending"}', name='retrieve_payment_status', tool_calls=None)]

2.5 Model Generate Final Answer

In [77]:
response = client.chat(model=model, messages=chat_history)
response.choices[0].message.content

'Your transaction with ID T1005 is currently pending. If you have any further questions or need assistance, feel free to ask!'